In [1]:
import pathlib
import pandas as pd
import json
import re
from tqdm import tqdm

## creation d'un dataframe à partir des fichiers CSV de Lorenzo

In [2]:
df_pan = pd.read_csv("./pan.csv").drop('Unnamed: 0', axis=1)
df_swt = pd.read_csv("./swt.csv").drop('Unnamed: 0', axis=1)
df_xqrs = pd.read_csv("./xqrs.csv").drop('Unnamed: 0', axis=1)

In [30]:
df_pan2 = pd.read_csv("./pan_v2.csv").drop('Unnamed: 0', axis=1)
df_swt2 = pd.read_csv("./swt_v2.csv").drop('Unnamed: 0', axis=1)
df_xqrs2 = pd.read_csv("./xqrs_v2.csv").drop('Unnamed: 0', axis=1)

In [31]:
#suppression des examen dont la durée est inférieure à 2min30 et dont la correlation SWT_XQRS est inférieure à 0.9
df_xqrs2 = df_xqrs2[(df_xqrs2['exam_duration'] > 151) & (df_xqrs2['SWT_vs_XQRS']>0.9)]
df_swt2 = df_swt2[(df_swt2['exam_duration'] > 151) & (df_swt2['SWT_vs_XQRS']>0.9)]

In [32]:
### définition des clés

df_xqrs["cle_exam"]= df_xqrs['Set']+"_"+df_xqrs['Categorie Montage'].astype(str)
df_xqrs["cle_exam"]= df_xqrs["cle_exam"] +"_"+df_xqrs['Patient'].astype(str)
df_xqrs["cle_exam"]= df_xqrs["cle_exam"] +"_"+df_xqrs['Session'].astype(str)
df_xqrs["cle_exam"]= df_xqrs["cle_exam"] +"_"+df_xqrs['File N°'].astype(str)


df_swt["cle_exam"]= df_swt['Set']+"_"+df_swt['Categorie Montage'].astype(str)
df_swt["cle_exam"]= df_swt["cle_exam"] +"_"+df_swt['Patient'].astype(str)
df_swt["cle_exam"]= df_swt["cle_exam"] +"_"+df_swt['Session'].astype(str)
df_swt["cle_exam"]= df_swt["cle_exam"] +"_"+df_swt['File N°'].astype(str)


df_xqrs2["cle_exam"]= df_xqrs2['Set']+"_"+df_xqrs2['Categorie Montage'].astype(str)
df_xqrs2["cle_exam"]= df_xqrs2["cle_exam"] +"_"+df_xqrs2['Patient'].astype(str)
df_xqrs2["cle_exam"]= df_xqrs2["cle_exam"] +"_"+df_xqrs2['Session'].astype(str)
df_xqrs2["cle_exam"]= df_xqrs2["cle_exam"] +"_"+df_xqrs2['File N°'].astype(str)


df_swt2["cle_exam"]= df_swt2['Set']+"_"+df_swt2['Categorie Montage'].astype(str)
df_swt2["cle_exam"]= df_swt2["cle_exam"] +"_"+df_swt2['Patient'].astype(str)
df_swt2["cle_exam"]= df_swt2["cle_exam"] +"_"+df_swt2['Session'].astype(str)
df_swt2["cle_exam"]= df_swt2["cle_exam"] +"_"+df_swt2['File N°'].astype(str)


In [9]:
list_features_non_linear = [
'csi',
'cvi',
'Modified_csi',
'sampen',
'sd1',
'sd2',
'ratio_sd2_sd1',
'label']

list_features_freq=[
'lf',
'hf',
'vlf',
'lf_hf_ratio']

list_features_temp=[
'mean_nni',
'sdnn',
'sdsd',
'nni_50',
'pnni_50',
'nni_20',
'pnni_20',
'rmssd',
'median_nni',
'range_nni',
'cvsd',
'cvnni',
'mean_hr',
'max_hr',
'min_hr',
'std_hr']

list_features = list_features_non_linear+list_features_freq+list_features_temp

list_features_control =[
    'mean_nni',
    'csi',
    'hf'
]

In [33]:
## calcul de la max duration de chaque examen et ajout dans une colonne
df_xqrs['max_duration']=df_xqrs.groupby('cle_exam')["interval_index"].transform('max')
df_swt['max_duration']=df_swt.groupby('cle_exam')["interval_index"].transform('max')
df_xqrs2['max_duration']=df_xqrs2.groupby('cle_exam')["interval_index"].transform('max')
df_swt2['max_duration']=df_swt2.groupby('cle_exam')["interval_index"].transform('max')

In [34]:
#calcul du % de remplissage des features pour chaque examen
for feat in list_features_control:
    df_xqrs2[feat+'%']=df_xqrs2[['cle_exam',feat]].groupby("cle_exam")[feat].transform('count')/df_xqrs2.groupby('cle_exam')["interval_index"].transform('count')*100
    df_swt2[feat+'%']=df_swt2[['cle_exam',feat]].groupby("cle_exam")[feat].transform('count')/df_swt2.groupby('cle_exam')["interval_index"].transform('count')*100
    

In [35]:
#suppression des examens qui ont moins de 15 lignes (et ou pourtant la durée d'examen est supérieure à 150secondes)
df_xqrs2= df_xqrs2[df_xqrs2['max_duration'] > 15]
df_swt2= df_swt2[df_swt2['max_duration'] > 15]

In [36]:
#suppression des examens où le "ht" est calculé dans moins de 40% des cas
df_xqrs2 = df_xqrs2[df_xqrs2['hf%'] >40]
df_swt2 = df_swt2[df_swt2['hf%'] >40]

In [37]:
## calcul du pourcentage de valeurs remplies pour chaque feature sur le dataset d'origine:

# pour XQRS:
df_result_xqrs=df_xqrs[['cle_exam','interval_index']].groupby("cle_exam").count()
for feat in list_features:
    df_result_xqrs[feat] = df_xqrs[['cle_exam',feat]].groupby("cle_exam").count()
    df_result_xqrs[feat+"%"] = df_result_xqrs[feat]/df_result_xqrs["interval_index"]*100
    df_result_xqrs = df_result_xqrs.drop(feat,axis=1)

#pour SWT:
df_result_swt=df_swt[['cle_exam','interval_index']].groupby("cle_exam").count()
for feat in list_features:
    df_result_swt[feat] = df_swt[['cle_exam',feat]].groupby("cle_exam").count()
    df_result_swt[feat+"%"] = df_result_swt[feat]/df_result_swt["interval_index"]*100
    df_result_swt = df_result_swt.drop(feat,axis=1)

# pour XQRS:
df_result_xqrs2=df_xqrs2[['cle_exam','interval_index']].groupby("cle_exam").count()
for feat in list_features:
    df_result_xqrs2[feat] = df_xqrs2[['cle_exam',feat]].groupby("cle_exam").count()
    df_result_xqrs2[feat+"%"] = df_result_xqrs2[feat]/df_result_xqrs2["interval_index"]*100
    df_result_xqrs2 = df_result_xqrs2.drop(feat,axis=1)

#pour SWT:
df_result_swt2=df_swt2[['cle_exam','interval_index']].groupby("cle_exam").count()
for feat in list_features:
    df_result_swt2[feat] = df_swt2[['cle_exam',feat]].groupby("cle_exam").count()
    df_result_swt2[feat+"%"] = df_result_swt2[feat]/df_result_swt2["interval_index"]*100
    df_result_swt2 = df_result_swt2.drop(feat,axis=1) 

In [38]:
### calcul des categories pour XQRS
df_result_xqrs['interval_index_cat'] = df_result_xqrs["interval_index"].apply(lambda x: "0 to 1min" if (x<7) 
                                                                    else "1 to 2min30" if x<16 
                                                                    else ">2min30" )

df_result_xqrs['csi%_cat'] = df_result_xqrs["csi%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )

df_result_xqrs['mean_nni%_cat'] = df_result_xqrs["mean_nni%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )

df_result_xqrs['lf%_cat'] = df_result_xqrs["lf%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )
### calcul des categories pour SWT

df_result_swt['interval_index_cat'] = df_result_swt["interval_index"].apply(lambda x: "0 to 1min" if (x<7) 
                                                                    else "1 to 2min30" if x<16 
                                                                    else ">2min30" )

df_result_swt['csi%_cat'] = df_result_swt["csi%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )

df_result_swt['mean_nni%_cat'] = df_result_swt["mean_nni%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )

df_result_swt['lf%_cat'] = df_result_swt["lf%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )

In [39]:
### calcul des categories pour XQRS nettoyé
df_result_xqrs2['interval_index_cat'] = df_result_xqrs2["interval_index"].apply(lambda x: "0 to 1min" if (x<7) 
                                                                    else "1 to 2min30" if x<16 
                                                                    else ">2min30" )

df_result_xqrs2['csi%_cat'] = df_result_xqrs2["csi%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )

df_result_xqrs2['mean_nni%_cat'] = df_result_xqrs2["mean_nni%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )

df_result_xqrs2['lf%_cat'] = df_result_xqrs2["lf%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )
### calcul des categories pour SWT nettoyé

df_result_swt2['interval_index_cat'] = df_result_swt2["interval_index"].apply(lambda x: "0 to 1min" if (x<7) 
                                                                    else "1 to 2min30" if x<16 
                                                                    else ">2min30" )

df_result_swt2['csi%_cat'] = df_result_swt2["csi%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )

df_result_swt2['mean_nni%_cat'] = df_result_swt2["mean_nni%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )

df_result_swt2['lf%_cat'] = df_result_swt2["lf%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                                    else "40 to 60" if x<60
                                                                    else "60 to 80" if x<80
                                                                    else "80 to 100" )

In [40]:
#nombre d'examen XQRS par catégorie de durée avant cleaning
df_result_xqrs[['interval_index_cat','interval_index']].groupby('interval_index_cat').count()

,interval_index
interval_index_cat,
0 to 1min,163
1 to 2min30,208
>2min30,4709


In [41]:
#nombre d'examen XQRS par catégorie de durée après cleaning
df_result_xqrs2[['interval_index_cat','interval_index']].groupby('interval_index_cat').count()

,interval_index
interval_index_cat,
>2min30,3268


In [42]:
#nombre d'examen SWT par catégorie de durée avant cleaning
df_result_swt[['interval_index_cat','interval_index']].groupby('interval_index_cat').count()

,interval_index
interval_index_cat,
0 to 1min,174
1 to 2min30,212
>2min30,4695


In [56]:
#nombre d'examen SWT par catégorie de durée après cleaning
df_result_swt2[['interval_index_cat','interval_index']].groupby('interval_index_cat').count()

,interval_index
interval_index_cat,
>2min30,3321


In [44]:
#nombre d'examen XQRS (interval_count) et nombre de lignes (index_sum) par taux de calcul du csi 
#(i.e il y a 733 examen pour lesquels le CSI est calcul entre 0 et 20% des lignes de l'examen)
df_result_xqrs[['csi%_cat','interval_index']].groupby('csi%_cat').agg(['count','sum'])

interval_index        
                   count     sum
csi%_cat                        
0 to 20              733   28511
20 to 40             239   10827
40 to 60             342   15497
60 to 80            1518   59779
80 to 100           2248  194331

In [45]:
#nombre d'examen XQRS (interval_count) et nombre de lignes (index_sum) par taux de calcul du csi 
#(i.e il y a 733 examen pour lesquels le CSI est calcul entre 0 et 20% des lignes de l'examen)
# APRES CLEANING
df_result_xqrs2[['csi%_cat','interval_index']].groupby('csi%_cat').agg(['count','sum'])

interval_index        
                   count     sum
csi%_cat                        
40 to 60              32    1814
60 to 80            1111   41300
80 to 100           2125  180848

In [46]:
#nombre d'examen SWT (interval_count) et nombre de lignes (index_sum) par taux de calcul du csi 
#(i.e il y a 733 examen pour lesquels le CSI est calcul entre 0 et 20% des lignes de l'examen)
# APRES CLEANING
df_result_swt[['csi%_cat','interval_index']].groupby('csi%_cat').agg(['count','sum'])

interval_index        
                   count     sum
csi%_cat                        
0 to 20              580   23021
20 to 40             213    8678
40 to 60             424   19386
60 to 80            1581   61070
80 to 100           2283  193367

In [47]:
#nombre d'examen SWT (interval_count) et nombre de lignes (index_sum) par taux de calcul du csi 
#(i.e il y a 733 examen pour lesquels le CSI est calcul entre 0 et 20% des lignes de l'examen)
# APRES CLEANING
df_result_swt2[['csi%_cat','interval_index']].groupby('csi%_cat').agg(['count','sum'])

interval_index        
                   count     sum
csi%_cat                        
20 to 40               1      66
40 to 60              33    2394
60 to 80            1116   39550
80 to 100           2171  184389

In [48]:
df_result_xqrs[['mean_nni%_cat','interval_index']].groupby('mean_nni%_cat').agg(['count','sum'])

interval_index        
                       count     sum
mean_nni%_cat                       
0 to 20                  177   10424
20 to 40                 254   13536
40 to 60                 248   12911
60 to 80                 356   20026
80 to 100               4045  252048

In [49]:
df_result_xqrs2[['mean_nni%_cat','interval_index']].groupby('mean_nni%_cat').agg(['count','sum'])

interval_index        
                       count     sum
mean_nni%_cat                       
40 to 60                   6     564
60 to 80                  65    5013
80 to 100               3197  218385

In [50]:
df_result_swt[['mean_nni%_cat','interval_index']].groupby('mean_nni%_cat').agg(['count','sum'])

interval_index        
                       count     sum
mean_nni%_cat                       
0 to 20                   82    6375
20 to 40                 194   12262
40 to 60                 290   14743
60 to 80                 427   23405
80 to 100               4088  248737

In [53]:
df_result_swt2[['mean_nni%_cat','interval_index']].groupby('mean_nni%_cat').agg(['count','sum'])

interval_index        
                       count     sum
mean_nni%_cat                       
40 to 60                   8     980
60 to 80                  59    4320
80 to 100               3254  221099

In [51]:
# xqrs LF AVANT
df_result_xqrs[['lf%_cat','interval_index']].groupby('lf%_cat').agg(['count','sum'])

interval_index        
                   count     sum
lf%_cat                         
0 to 20             1189   43563
20 to 40             273   13830
40 to 60            1316   49819
60 to 80            1603  106983
80 to 100            699   94750

In [54]:
# xqrs LF APRES
df_result_xqrs2[['lf%_cat','interval_index']].groupby('lf%_cat').agg(['count','sum'])

interval_index       
                   count    sum
lf%_cat                        
40 to 60            1132  39218
60 to 80            1489  98016
80 to 100            647  86728

In [52]:
#swt LF avant
df_result_swt[['lf%_cat','interval_index']].groupby('lf%_cat').agg(['count','sum'])

interval_index        
                   count     sum
lf%_cat                         
0 to 20              972   33470
20 to 40             271   12564
40 to 60            1450   54854
60 to 80            1688  110457
80 to 100            700   94177

In [55]:
##swt LF après
df_result_swt2[['lf%_cat','interval_index']].groupby('lf%_cat').agg(['count','sum'])

interval_index       
                   count    sum
lf%_cat                        
40 to 60            1139  38595
60 to 80            1499  96264
80 to 100            683  91540

In [57]:
df_xqrs2.to_csv('./xqrs_v2_cleaned.csv')

In [58]:
df_swt2.to_csv('./swt_v2_cleaned.csv')